In [1]:
!pip install pandas
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [62]:
import pandas as pd
import numpy as np
# Подгружаем датасет
data = pd.read_csv('BankChurners.csv', index_col='CLIENTNUM')

# Удаляем строки с NA
data = data.dropna()
# Преобразуем поле Credit_Limit: 0 при >= 10000, 1 при < 10000
data['Credit_Limit'] = np.where(data['Credit_Limit'] >= 10000, 0, 1)
# Удаляем строки с unknown
data.drop(data.loc[data['Education_Level']=='Unknown'].index, inplace=True)
data.drop(data.loc[data['Marital_Status']=='Unknown'].index, inplace=True)
data.drop(data.loc[data['Income_Category']=='Unknown'].index, inplace=True)

In [63]:
# Отделяем Credit_Limit от остальных данных
Credit_Limit = data.loc[:, data.columns.isin(['Credit_Limit'])]
X = data.loc[:, data.columns.isin(['Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count', 
                                   'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category', 
                                   'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 
                                   'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 
                                   'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 
                                   'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 
                                   'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'])]

In [64]:
# Переводим категориальные переменные в One hot encoding
education = pd.get_dummies(X.Education_Level, prefix='Education')
marital = pd.get_dummies(X.Marital_Status, prefix='Marital')
flag = pd.get_dummies(X.Attrition_Flag, prefix='Flag')
gender = pd.get_dummies(X.Gender, prefix='Gender')
income = pd.get_dummies(X.Income_Category, prefix='Income')
card = pd.get_dummies(X.Card_Category, prefix='Card')
X = X.drop(columns=['Education_Level','Marital_Status', 'Attrition_Flag', 'Gender', 'Income_Category', 'Card_Category'])

In [65]:
# Добавляем датафреймы eho переменных к основному датафрейму
frames = [X, education, marital, flag, gender, income, card]
X = pd.concat(frames, axis=1)
X = X.drop(columns=['Education_College', 'Gender_M', 'Card_Gold', 'Marital_Divorced', 'Flag_Attrited Customer', 'Income_Less than $40K'])
X

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,...,Marital_Single,Flag_Existing Customer,Gender_F,Income_$120K +,Income_$40K - $60K,Income_$60K - $80K,Income_$80K - $120K,Card_Blue,Card_Platinum,Card_Silver
CLIENTNUM,,,,,,,,,,,,,,,,,,,,,
768805383,45,3,39,5,1,3,777,11914.0,1.335,1144,...,0,1,0,0,0,1,0,1,0,0
818770008,49,5,44,6,1,2,864,7392.0,1.541,1291,...,1,1,1,0,0,0,0,1,0,0
713982108,51,3,36,4,1,0,0,3418.0,2.594,1887,...,0,1,0,0,0,0,1,1,0,0
709106358,40,3,21,5,1,0,0,4716.0,2.175,816,...,0,1,0,0,0,1,0,1,0,0
713061558,44,2,36,3,1,2,1247,2763.0,1.376,1088,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710841183,54,1,34,5,2,0,2109,11831.0,0.660,15577,...,1,1,0,0,0,1,0,1,0,0
713899383,56,1,50,4,1,4,606,3082.0,0.570,14596,...,1,1,1,0,0,0,0,1,0,0
772366833,50,2,40,3,2,3,1851,2152.0,0.703,15476,...,1,1,0,0,1,0,0,1,0,0


In [66]:
# Разделяем датасет на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Credit_Limit, test_size = 0.33, random_state=42)

In [67]:
# Построим классификатор типа решающее дерево (decision tree classifier)
from sklearn import tree

dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)

In [68]:
# Оценим точность классификатора с помощью метрик precision, recall и F1 на тестовой выборке
from sklearn.model_selection import cross_val_score

print("f1:", str(np.average(cross_val_score(dtc, X_test, y_test, scoring='f1'))))
print("precision:", str(np.average(cross_val_score(dtc, X_test, y_test, scoring='precision'))))
print("recall:", str(np.average(cross_val_score(dtc, X_test, y_test, scoring='recall'))))

f1: 0.9965350958186002
precision: 0.9965533328315231
recall: 0.9959537572254333


In [69]:
# Построим классификатор типа случайный лес (random forest classifier)
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
# Используем методы .values.ravel(), чтобы привести y_train к нужному формату (избежать warning)
rfc = rfc.fit(X_train, y_train.values.ravel())

In [70]:
# Оценим точность классификатора с помощью метрик precision, recall и F1 на тестовой выборке

# Используем методы .values.ravel(), чтобы привести y_test к нужному формату (избежать warning)
print("f1:", str(np.average(cross_val_score(rfc, X_test, y_test.values.ravel(), scoring='f1'))))
print("precision:", str(np.average(cross_val_score(rfc, X_test, y_test.values.ravel(), scoring='precision'))))
print("recall:", str(np.average(cross_val_score(rfc, X_test, y_test.values.ravel(), scoring='recall'))))

f1: 0.9950879406672952
precision: 0.9931131765267592
recall: 0.9959537572254336


In [71]:
df = pd.DataFrame(columns=['Метрика', 'Decision Tree', 'Random Forest'])
df = df.append({'Метрика':'f1', 'Decision Tree': '0.996', 'Random Forest': '0.994'}, ignore_index=True)
df = df.append({'Метрика':'precision', 'Decision Tree': '0.997', 'Random Forest': '0.991'}, ignore_index=True)
df = df.append({'Метрика':'recall', 'Decision Tree': '0.996', 'Random Forest': '0.995'}, ignore_index=True)
df

,Метрика,Decision Tree,Random Forest
0,f1,0.996,0.994
1,precision,0.997,0.991
2,recall,0.996,0.995


Классификатор Decision Tree имеет лучший результат